# 走行

このNotebookで自動走行をおこないます。

## Jetsonの認識

In [ ]:
import Jetson.GPIO as GPIO

BOARD_NAME=GPIO.gpio_pin_data.get_data()[0]
if BOARD_NAME == "JETSON_NX":
    print("Jetson Xavier NXを認識")
    I2C_BUSNUM = 8
    MODE = 2
elif BOARD_NAME == "JETSON_XAVIER":
    print("Jetson AGX Xavierを認識")
    I2C_BUSNUM = 8
    MODE = 2
elif BOARD_NAME == "JETSON_NANO":
    print("Jetson NANOを認識")
    I2C_BUSNUM = 1
    MODE = 0

In [ ]:
# Orin Nanoは7を使用
I2C_BUSNUM = 7

In [ ]:
!echo "jetson" | sudo -S nvpmodel -m $MODE

In [ ]:
!echo "jetson" | sudo -S nvpmodel -q

In [ ]:
!echo "jetson" | sudo -S jetson_clocks

## ログの表示用 Widget

In [ ]:
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
import os
import glob

l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value

## PWMの値の読み込み

In [ ]:
import Fabo_PCA9685
import time
import pkg_resources
import smbus
import time
import json

SMBUS='smbus'
BUSNUM=I2C_BUSNUM
SERVO_HZ=60
INITIAL_VALUE=300
bus = smbus.SMBus(BUSNUM)
PCA9685 = Fabo_PCA9685.PCA9685(bus,INITIAL_VALUE,address=0x40)
PCA9685.set_hz(SERVO_HZ)

STEERING_CH = 0
THROTTLE_CH = 1
direction = 0

pwm_front = 0
pwm_back = 0

with open('pwm_params.json') as f:
    json_str = json.load(f)
    
    pwm_stop = json_str["pwm_speed"]["stop"]
    pwm_front = json_str["pwm_speed"]["front"]
    pwm_back = json_str["pwm_speed"]["back"]
    pwm_left = json_str["pwm_steering"]["left"]
    pwm_center = json_str["pwm_steering"]["center"]
    pwm_right = json_str["pwm_steering"]["right"]

    
if pwm_front >= pwm_back:
    direction = 0
else:
    direction = 1
    
PCA9685.set_channel_value(STEERING_CH, pwm_center)
PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)

## カメラの読込

この部分でエラーが発生する場合は、Jetsonの再起動をお願いします。<br>
それでも、カメラが認識できない場合は、ケーブルの接続確認をしてください。


In [ ]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=30)

## 走行処理

In [ ]:
import torch
from torch import nn
from torchvision.models import resnet18
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import math

IMG_WIDTH=224
IMG_HEIGHT=224

# この部分は、画像から情報を取り出すためのものです。名前が「抽出器」であるように、これは画像から重要な特徴を見つけ出すために使います。
class ImageFeatureExtractor(nn.Module):
    """Image Feature Extractor using ResNet18"""
    def __init__(self, output_size, dropout=0.1):
        super().__init__()
        # ResNet18は事前に訓練された大きなネットワークで、画像から特徴を抽出するのにとても役立ちます。
        self.resnet = resnet18(pretrained=True)
        # ここではネットワークの最後に新しい部分を追加しています。これは特徴を取り出すためのものです。
        self.resnet.fc = nn.Sequential(
          nn.Linear(self.resnet.fc.in_features, output_size),
          nn.ReLU(inplace=True),
          nn.Dropout(dropout),
        )

    # この部分は、画像を取り、それを特徴抽出器に通すことで特徴を取り出します。
    def forward(self, x):
        return self.resnet(x)

# この部分は、情報を処理するためのものです。これは「Transformer」部分と呼ばれ、順序を考慮しながら情報を分析します。
class TransformerBlock(nn.Module):
    """Transformer Block for processing sequence data."""
    def __init__(self, input_size, nhead, nhid, nlayers, dropout=0.1):
        super().__init__()
        # 位置エンコーディングは、順序を覚えるための特別な技術です。
        self.pos_encoder = PositionalEncoding(input_size, dropout)
        # この部分は実際の「Transformer」です。これが情報を処理します。
        encoder_layers = TransformerEncoderLayer(input_size, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)

    # この部分は、情報を「Transformer」に通すことで情報を処理します。
    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.pos_encoder(x)
        return self.transformer_encoder(x)

# この部分は、上記のすべてを組み合わせて全体のモデルを形成します。画像から特徴を取り出し、それを「Transformer」に通すことで情報を処理します。
class JetFormerModel(nn.Module):
    """Main model that combines ImageFeatureExtractor and TransformerBlock."""
    def __init__(self, ninp, nhead, nhid, nlayers, dropout=0.1):
        super(JetFormerModel, self).__init__()
        # 画像から特徴を抽出する部分
        self.img_feature_extractor = ImageFeatureExtractor(ninp//2, dropout)
        # 以前のハンドルのペアを処理する部分
        self.previous_handle_fc = nn.Sequential(
            nn.Linear(2, ninp//2),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
        )
        # 「Transformer」部分
        self.transformer_block = TransformerBlock(ninp, nhead, nhid, nlayers, dropout)
        # 最後に、結果を得るための部分
        self.fc = nn.Sequential(
            nn.Linear(ninp, 64),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(64, 2),
        )

    # この部分は、すべての部品を一緒に組み立てて結果を得る部分です。
    def forward(self, images, previous_handle_pairs):
        img_features = self.img_feature_extractor(images)
        previous_handle_features = self.previous_handle_fc(previous_handle_pairs)
        out = torch.cat([img_features, previous_handle_features], dim=-1)
        out = self.transformer_block(out)
        out = self.fc(out[:, 0, :])
        return out

# 位置エンコーディングは、情報がどの順序で来たかを覚えるためのものです。
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
      x = x + self.pe[:x.size(0), :]
      return self.dropout(x)


In [ ]:
# 必要なライブラリやツールを読み込みます
import os
import cv2
import datetime
import os
import torch
import torchvision.transforms as transforms
import cv2
import PIL.Image
import numpy as np

batch_size = 128
num_epochs = 500
nhid = 1024
ninp = 512
nhead = 4
nlayers = 4
learning_rate = 0.00006
# 「ディープラーニング」のためのモデル（計算の設計図）を読み込みます
# 「ディープラーニング」はコンピュータに大量のデータから学習させる技術の一つです
device = torch.device('cuda')
model = JetFormerModel(ninp=ninp, nhead=nhead, nhid=nhid, nlayers=nlayers).to(device)

# 学習した結果（重みやバイアス）を読み込みます
# 「重み」や「バイアス」はモデルの中の計算に使われるパラメータ（数値）で、これらの値によりモデルの出力結果が変わります
checkpoint = torch.load('./model_checkpoint_0801_170.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()  # モデルを評価モードに切り替えます
load = True
# 画像の色の強さを調整するための値を定義します
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

# 画像を処理するための関数を定義します
def preprocess(image):
    device = torch.device('cuda')
    image = PIL.Image.fromarray(image)  # 画像をPIL形式に変換します
    image = transforms.functional.to_tensor(image).to(device)  # 画像をテンソル（多次元配列）に変換します
    image.sub_(mean[:, None, None]).div_(std[:, None, None])  # 画像の色の強さを調整します
    return image[None, ...]

In [ ]:
import torch
import threading
#from utils import preprocess
import subprocess
import cv2
import time
#from torch2trt import TRTModule
import subprocess
import datetime


model_widget = ipywidgets.Dropdown(options=[],description='モデル')
model_time_widget = ipywidgets.Label(description='作成日時')
load_button = ipywidgets.Button(description='Load')
run_button = ipywidgets.Button(description='Run')
stop_button = ipywidgets.Button(description='Stop')
pwm_left_widget = ipywidgets.IntText(value=pwm_left,description='PWM 左')
pwm_center_widget = ipywidgets.IntText(value=pwm_center,description='PWM 中央')
pwm_right_widget = ipywidgets.IntText(value=pwm_right,description='PWM 右')
check_left_button = ipywidgets.Button(description='チェック左')
check_center_button = ipywidgets.Button(description='チェック中央')
check_right_button = ipywidgets.Button(description='チェック右')
pwm_stop_widget = ipywidgets.IntText(value=pwm_stop,description='PWM 停止')
name_widget = ipywidgets.Text(description='映像の保存先')
record_box = ipywidgets.Checkbox(False, description='録画')
speed_high_slider = ipywidgets.IntSlider(description='High', min=0, max=100, step=1, value=20, orientation='horizontal')
speed_low_slider = ipywidgets.IntSlider(description='Low', min=0, max=100, step=1, value=10, orientation='horizontal')
steering_gain_slider = ipywidgets.FloatSlider(description='Steering gain', min=0.1, max=2.0, step=0.1, value=1.0, orientation='horizontal')

record = False
running = False
load = False

def map_rc(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) // (in_max - in_min) + out_min

def handle(x):
    global pwm_right,pwm_left,STEERING_CH,PCA9685
    x = map_rc(x, 224, 0, pwm_right, pwm_left)
    PCA9685.set_channel_value(STEERING_CH, x)
    
def live():
    global device, process, model,running,camera,IMG_WIDTH,record,model_trt,preprocess,pwm_stop,save_dir,num,cv2,count,speed_high_slider,speed_low_slider,steering_gain_slider
    
    count = 1
    num = 1
    xy_sequence = torch.tensor([[(112/IMG_WIDTH - 0.5) * 2, (112/IMG_HEIGHT - 0.5) * 2]], device=device).float()  # 初期の位置を決めます

    while running:
        image = camera.read()
        if record == True:
            remarked_img = image.copy()
            
        preprocessed = preprocess(image)  # 画像を処理します
        preprocessed = preprocessed.to(device)  # 画像をGPUに送ります
        output = model(preprocessed, xy_sequence)  # モデルに画像を入力して結果をもらいます
        
        output = output.detach().cpu().numpy().flatten()  # 結果を使いやすい形に変換します
        xy_sequence = torch.tensor([output], device=device).float()  # 次の画像で使うために、結果を保存します

        result_x = output[0]  # 結果からx座標（左右の位置）を取り出します
        result_speed = output[1]  # 結果から速度を取り出します
        
        x = int(IMG_WIDTH * (result_x / 2.0 + 0.5))
        #y = int(IMG_WIDTH * (y / 2.0 + 0.5))
        speed = int(IMG_WIDTH * (result_speed / 2.0 + 0.5))
        handle(x)
        
        if direction == 0:
            pwm_speed_high = pwm_stop + speed_high_slider.value
            pwm_speed_low = pwm_stop + speed_low_slider.value
        else:
            pwm_speed_high = pwm_stop - speed_high_slider.value
            pwm_speed_low = pwm_stop - speed_low_slider.value
        #PCA9685.set_channel_value(THROTTLE_CH, pwm_speed)
        
        if speed > 112:
            PCA9685.set_channel_value(THROTTLE_CH, pwm_speed_high)
        elif speed <= 112:
            PCA9685.set_channel_value(THROTTLE_CH, pwm_speed_low)

        if record == True:
            if num % 2 == 0:
                name = "0_0_{:0=5}.jpg".format(num)
                image_path = os.path.join(save_dir, name)
                cv2.imwrite(image_path, remarked_img)
            num += 1
        count += 1
        
        #write_log("Count:" + str(count))
        
def run(change):
    global running,execute_thread,name_widget,save_dir,start_time,load,speed_high_slider,speed_low_slider,steering_gain_slider
    
    if load == False:
        write_log("モデルが読み込まれていません")
        return

    if running == False:
        if record == True:
            if name_widget.value != "":
                save_dir = "run/" + name_widget.value + "/xy/" 
                if not os.path.exists(save_dir):
                    subprocess.call(['mkdir', '-p', save_dir])
                write_log(save_dir + "にデータを保存します。")            

                running = True
                execute_thread = threading.Thread(target=live)
                execute_thread.start()
                start_time = time.time()
                write_log("AIが起動しました。")
                if direction == 0:
                    pwm_speed_high = pwm_stop + speed_high_slider.value
                    pwm_speed_low = pwm_stop + speed_low_slider.value
                else:
                    pwm_speed_high = pwm_stop - speed_high_slider.value
                    pwm_speed_low = pwm_stop - speed_low_slider.value
                write_log("Steering Gain:" + str(steering_gain_slider.value) + " Speed High:" + str(pwm_speed_high) + " Low:" + str(pwm_speed_low) + "で走行開始します。")
            else:
                write_log("映像の保存先を入力してください。")
        else:
            running = True
            execute_thread = threading.Thread(target=live)
            execute_thread.start()
            start_time = time.time()
            write_log("AIが起動しました。") 
            if direction == 0:
                pwm_speed_high = pwm_stop + speed_high_slider.value
                pwm_speed_low = pwm_stop + speed_low_slider.value
            else:
                pwm_speed_high = pwm_stop - speed_high_slider.value
                pwm_speed_low = pwm_stop - speed_low_slider.value
            write_log("Steering Gain:" + str(steering_gain_slider.value) + " Speed High:" + str(pwm_speed_high) + " Low:" + str(pwm_speed_low) + "で走行開始します。")
        
    
def stop(change):
    global running,execute_thread,end_time,start_time,count,pwm_stop
    if running == True:
        PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
        try:
            end_time = time.time() - start_time
            fps = count/int(end_time)
            process_time = int((end_time/count)*1000)
        except:
            fps = -1
            process_time = -1
        
        write_log("AIを停止しました。")
        write_log("処理結果:FPS: " + str(round(fps,2)) + ",処理回数: " + str(count) + ",　処理時間(1回平均値): " + str(process_time) + " ms")
        running = False
        execute_thread.join()
    else:
        PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
        write_log("現在AIは動いていません。")

def load_model(change):
    global model_trt,model_widget,load
    try:
        #write_log(model_widget.value + "の読込を実行します(初回は時間がかかります)。")
        #model_trt = TRTModule()
        #model_trt.load_state_dict(torch.load(model_widget.value))
        #write_log(model_widget.value + "の読込に成功しました。")
        load = True
    except:
        write_log("[Error]")
        
def model_list(change):
    global model_widget
    try:
        files = glob.glob('./model_trt/*.pth', recursive=True)
        model_widget.options = files
        ts = os.path.getctime(files[0])
        d = datetime.datetime.fromtimestamp(ts)
        s = d.strftime('%Y-%m-%d %H:%M:%S')
        model_time_widget.value = s
    except:
        model_widget.options = []
model_list("list")

def change_file(change):
    file = model_widget.value
    ts = os.path.getctime(file)
    d = datetime.datetime.fromtimestamp(ts)
    s = d.strftime('%Y-%m-%d %H:%M:%S')
    model_time_widget.value = s
model_widget.observe(change_file, names='value')

def on_video(change):
    global record
    record^=True
    
load_button.on_click(load_model)
run_button.on_click(run)
stop_button.on_click(stop)
record_box.observe(on_video)

走行までの流れは以下の通りです。

1. <b>学習済みモデルを指定してLoadする</b><br>
2. <b>Steering gainの値を調整する</b><br>
初期値は1.0です。0.1〜2.0の値で設定できる、1.0より大きな値にするとハンドルのキレ角が鋭くなる<br>
3. <b>High, Lowの値を調整する</b><br>
4. <b>[オプション] 走行動画を録画する場合は、録画にチェックマークをいれて、保存ファイル名を指定する</b><br>
./runフォルダに保存<br>
5. <b>runボタンを押して、プロポの裏側のボタンを押して AIモードで自動走行開始する</b><br>
6. <b>終了時は、stopボタンを押す</b><br>
録画のチェックマークがついている場合は、stopで録画も終了<br>
<br>
カメラが60fpsで動いている場合は16ms以内、カメラが30fpsで動いている場合は、33ms以内での処理完了が正常な挙動となります。

In [ ]:
data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([model_widget,model_time_widget,load_button]),
    ipywidgets.HBox([steering_gain_slider, speed_high_slider, speed_low_slider]),
    ipywidgets.HBox([record_box,name_widget,run_button, stop_button]),
    process_widget
])
display(data_collection_widget)

### カメラの終了処理(必須)

In [ ]:
import time

camera.running = False
camera.cap.release()